In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import pandas as pd
device = torch.device("cuda")

from train_autoencoders import*
from coupled_learning_mlp import*
from GAN import*
from Mlp import*
from Testing_function import*
from Autoencoders import*
from train_mapping_functions import*
from coupled_learning_new import*

from torch.utils.data import Dataset, DataLoader

class MovieDataset(Dataset):
    def __init__(self, movies):
        self.movies = movies

    def __len__(self):
        return len(self.movies)

    def __getitem__(self, index):
        return index, self.movies[index, :]

class UserDataset(Dataset):
    def __init__(self, users):
        self.users = users

    def __len__(self):
        return self.users.shape[0]

    def __getitem__(self, index):
        return index, self.users[index, :]

In [2]:
df=pd.read_csv('/home/alexgi/Workspace/REC_SYSTEMS/My_codes/Data/Rs_no3.csv')
Movies_source=df.values       # movies x users
del df
df=pd.read_csv('/home/alexgi/Workspace/REC_SYSTEMS/My_codes/Data/Rt_no3.csv')
Movies_target=df.values   

Movies_source=Movies_source/5
Movies_target=Movies_target/5
print('source domain sparsity:', np.count_nonzero(Movies_source==0)/(Movies_source.shape[0]*Movies_source.shape[1])*100)
print('target domain sparsity:', np.count_nonzero(Movies_target==0)/(Movies_target.shape[0]*Movies_target.shape[1])*100)

Movies_source = Movies_source[:,1:Movies_source.shape[1]]
Movies_target = Movies_target[:,1:Movies_target.shape[1]]

source domain sparsity: 99.35004475168513
target domain sparsity: 98.77889420673183


In [3]:
#SIMPLE METHOD

batch_size = 32

layer_1_dim = 64
layer_2_dim = 32
layer_3_dim = 16


frmse = []
fmae = []

for i in range(1):

   Movies_source_train, Movies_source_test, Movies_target_train, Movies_target_test = train_test_split(Movies_source, Movies_target, test_size=0.2)
   dataset = (torch.from_numpy((np.concatenate((Movies_source_train, Movies_source_test)))))
   dataloader_train_source = DataLoader(dataset, batch_size, shuffle=False)

   dataset = (torch.from_numpy(Movies_target_train))
   dataloader_train_target = DataLoader(dataset, batch_size, shuffle=False)

   dataset = TensorDataset(torch.from_numpy(Movies_source_train), torch.from_numpy(Movies_target_train))
   dataloader_Coupled = DataLoader(dataset, batch_size, shuffle=False)

   # ============ Train Autoencoders ==============
   model_encoder_s = encoder(Movies_source_train.shape[1], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
   model_decoder_s = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Movies_source_train.shape[1]).cuda()

   model_encoder_t = encoder(Movies_target_train.shape[1], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
   model_decoder_t = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Movies_target_train.shape[1]).cuda()

   noise_level = 0
   learning_rate = 1e-3
   l2_w = 1e-8
   num_epochs = 30

   train_autoencoders(model_encoder_s, model_decoder_s, num_epochs,
                                                    learning_rate, dataloader_train_source, device, l2_w, noise_level)
   train_autoencoders(model_encoder_t, model_decoder_t, num_epochs,
                                                    learning_rate, dataloader_train_target, device, l2_w, noise_level)


   # #GAN_based 

   generator = Generator(input_size=16, hidden_size=32, output_size=16).to(device)
   discriminator = Discriminator(input_size=16, hidden_size=32).to(device)
   train_GAN(generator, discriminator, model_encoder_s, model_encoder_t, dataloader_Coupled, device, epochs=15, learning_rate=1e-03)
   # r3,m3 = test_data(model_encoder_s, generator, model_decoder_t, Movies_source_test, Movies_target_test, 0)
   # print('Before Coupled Learning')
   # print(r3, m3)
   
   
   # ============ Multilayer Percepton Training ==============
   # num_epochs = 10
   # learning_rate = 1e-3
   # l2_w = 1e-5
   # generator = mlp_network(layer_3_dim, 32).cuda()
   # train_MLP(generator, model_encoder_s, model_encoder_t, num_epochs, learning_rate, dataloader_Coupled, device, l2_w)
   # r3,m3=test_data(model_encoder_s, generator, model_decoder_t, Movies_source_test, Movies_target_test)
   # #print(r3, m3)
   
   
   
   # ============ Coupled =====================================
   # print('After Coupled Learning')
   num_epochs = 50
   learning_rate = 1e-4
   l2_w = 1e-8
   rmse_loss, mae_loss, precision, recall = train_Coupled_model(generator, model_encoder_s, model_decoder_t,
                                                                num_epochs, learning_rate, dataloader_Coupled,
                                                               device, Movies_source_test, Movies_target_test, l2_w, noise_level)
   # print(rmse_loss, mae_loss )
   
   
   frmse.append(rmse_loss)
   fmae.append(mae_loss)


print(np.mean(frmse), np.mean(fmae), precision, recall)

0.1979522556066513 0.155105859041214 0.8669704501518916 0.97970173985087


In [ ]:
#MATRIX_FACTORIZATION

mf_rmse = []
mf_mae = []

for i in range(1):

    #Load Training and Testing data

    Movies_source_train, Movies_source_test, Movies_target_train, Movies_target_test = train_test_split(Movies_source, Movies_target, test_size=0.2)
    Users_source_train = Movies_source_train.T
    Users_source_test  = Movies_source_test.T

    Users_target_train = Movies_target_train.T
    Users_target_test  = Movies_target_test.T

    Users_source=Movies_source.T
    # del df,Movies_s,Movies_t

    num_epochs = 10
    learning_rate = 1e-3
    l2_w = 1e-5

    batch_size_m = 500
    batch_size_u = 500
    batch_size = batch_size_m

    layer_1_dim = 64
    layer_2_dim = 32
    layer_3_dim = 16


 

    # Dataloader source domain
    dataset = (torch.tensor(Movies_source_train))
    dataloader_movies = DataLoader(dataset, batch_size_m, shuffle=False)
    dataset = (torch.tensor(Users_source_train))
    dataloader_users = DataLoader(dataset, batch_size_u, shuffle=False)

    model_encoder_m_s = encoder(Users_source_train.shape[0],layer_1_dim, layer_2_dim, layer_3_dim).cuda()
    model_decoder_m_s = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Users_source_train.shape[0]).cuda()

    model_encoder_u_s = encoder(Movies_source_train.shape[0], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
    model_decoder_u_s = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Movies_source_train.shape[0]).cuda()

    train_latentfactors_autoencoders(model_encoder_m_s, model_decoder_m_s, model_encoder_u_s, model_decoder_u_s, num_epochs,
                                    learning_rate, dataloader_movies, dataloader_users, device, l2_w,  batch_size_u)

    del dataset,dataloader_movies,dataloader_users

    #Dataloader Target domain --------------------------------------------------------------------------------------------------------

    dataset = (torch.tensor(Movies_target_train))
    dataloader_movies = DataLoader(dataset, batch_size_m, shuffle=False)
    dataset = (torch.tensor(Users_target_train))
    dataloader_users = DataLoader(dataset, batch_size_u, shuffle=False)

    model_encoder_m_t = encoder(Users_target_train.shape[0], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
    model_decoder_m_t = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Users_target_train.shape[0]).cuda()

    model_encoder_u_t = encoder(Movies_target_train.shape[0], layer_1_dim, layer_2_dim, layer_3_dim).cuda()
    model_decoder_u_t = decoder(layer_3_dim, layer_2_dim, layer_1_dim, Movies_target_train.shape[0]).cuda()

    train_latentfactors_autoencoders(model_encoder_m_t, model_decoder_m_t, model_encoder_u_t,
                                    model_decoder_u_t, num_epochs, learning_rate, dataloader_movies,
                                    dataloader_users, device, l2_w,  batch_size_u)

    del dataset, dataloader_movies, dataloader_users

    #Dataloader for the mapping MLP ----------------------------------------------------------------------------------------------------
    dataset = TensorDataset(torch.tensor(Movies_source_train), torch.tensor(Movies_target_train))
    dataloader_Coupled = DataLoader(dataset, batch_size, shuffle=False)
    num_epochs = 50
    learning_rate = 1e-3
    l2_w = 1e-5
    
    generator = Generator(input_size=16, hidden_size=32, output_size=16).to(device)
    discriminator = Discriminator(input_size=16, hidden_size=32).to(device)
    train_GAN(generator, discriminator, model_encoder_m_s, model_encoder_m_t, dataloader_Coupled, device, epochs=10, learning_rate= 1e-3)

    # generator = mlp_network(layer_3_dim, 16).cuda()
    # train_MLP(generator, model_encoder_m_s, model_encoder_m_t, num_epochs,
    #                       learning_rate, dataloader_Coupled, device, l2_w)

    # r3,m3 = test_data_Matrix_factorization(model_encoder_m_s, generator, model_encoder_u_t,
    #                   Movies_source_test, Movies_target_test, Users_target_train)

    del dataset, dataloader_Coupled
    #print("before Coupling")
    # print(r3, m3)

    # Coupled Model --------------------------------------------------------------------------------------------------------------------
    dataset = TensorDataset(torch.tensor(Movies_source_train), torch.tensor(Movies_target_train))
    dataloader_movies = DataLoader(dataset, batch_size_m, shuffle=False)
    dataset = (torch.tensor(Users_target_train))
    dataloader_users = DataLoader(dataset, batch_size_u, shuffle=False)

    num_epochs = 100
    learning_rate = 1e-3
    l2_w = 1e-8
    loss_rmse, loss_mae, prec, recall = train_Coupled_model_Matrix_factorization(generator, model_encoder_m_s, model_encoder_u_t, num_epochs, learning_rate,
                                              dataloader_movies, dataloader_users, device, Movies_source_test, Movies_target_test,
                                              l2_w, Users_target_train, Movies_target_train, batch_size_m, batch_size_u)
    mf_rmse.append(loss_rmse)
    mf_mae.append(loss_mae)

print(np.mean(mf_rmse), np.mean(mf_mae), prec, recall)

Count the sparsity levels

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/DATA_Recommendation/Rs_no4.csv')
Movies_source=df.values       # movies x users
df=pd.read_csv('/content/drive/MyDrive/DATA_Recommendation/Rt_no4.csv')
Movies_target=df.values

Movies_source_train, Movies_source_test, Movies_target_train, Movies_target_test = train_test_split(Movies_source, Movies_target, test_size=0.2)



n = 5000
index = np.random.choice(Movies_source.shape[1], n, replace=False)
Rsource = Movies_source
#Rsource[:,index] = 0
np.count_nonzero(Rsource)/(Rsource.shape[0]*Rsource.shape[1])*100


index = np.random.choice(Movies_target.shape[1], n, replace=False)
Rtarget = Movies_target
#Rtarget[:,index] = 0
np.count_nonzero(Rtarget)/(Rtarget.shape[0]*Rtarget.shape[1])*100

np.amax(Rsource)


pd.DataFrame(Rsource).to_csv("/content/drive/MyDrive/DATA/Rs_20.csv")

pd.DataFrame(Rtarget).to_csv("/content/drive/MyDrive/DATA/Rt_20.csv")